In [20]:
import time
import datetime
import pymongo

import pandas as pd
from dateutil.parser import parse

In [4]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient['1001']
tracklist_collection = db['tracklist_docs']

In [12]:
tracklist_date_hash = {}
for page in tracklist_collection.find({}):
    tracklist_date_hash[page['url']] = parse(page['created'])

In [13]:
print(len(list(tracklist_date_hash.keys())))

73105


In [83]:
count = 0
artist_counts = {}
for track in db['sequential_docs_fixed'].find({}):
    
    count += 1
    if count % 100000 == 0: print(count)
    artist_counts[track['track1_artist']] = artist_counts.get(track['track1_artist'], 0) + 1
    artist_counts[track['track2_artist']] = artist_counts.get(track['track2_artist'], 0) + 1


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000


In [ ]:
print(len(ordered_artists))
ordered_artists

In [ ]:
import pytz

def objective_function(array):
    
    obj = 0
    array = array[::-1]
    for i in range(len(array)-1):
        for j in range(i,len(array)):
             obj += array[i] - array[j]
    return obj


velocity_docs = []
ordered_artists = sorted(list(artist_counts.items()), key=lambda x: x[1], reverse=True)
for artist, count in ordered_artists[:]:
    
    print(artist, count)
    artist_docs = []
    
    for track in db['sequential_docs_fixed'].find({}):
        
        if track['track1_artist'] == artist or track['track2_artist'] == artist:

            artist_docs.append(\
                              {
                              'artist': artist,
                              'url': track['url'],
                              'date': tracklist_date_hash.get(track['url'], None),
                              }
                            )
   
    artist_df = pd.DataFrame(artist_docs)
    artist_df = artist_df.drop_duplicates()
    artist_df.dropna(inplace=True)
    artist_df.date = artist_df.apply(lambda x: x['date'].replace(tzinfo=pytz.UTC), axis=1)
    print(len(artist_df))
    
    artist_df.index = artist_df.date
    artist_df = artist_df.resample('M').count()
    artist_df.ffill(inplace=True)
    print(len(artist_df))
    
    artist_df['pct_change'] = artist_df['date'].pct_change()
    artist_df['smooth'] = artist_df['date'].rolling(window=6).mean()
    artist_df['artist_velocity'] = artist_df['smooth'].rolling(window=6)\
                                                      .apply(lambda x: objective_function(x))
    velocity_docs.extend(artist_df.to_dict('records'))

In [106]:
print(len(velocity_docs))

1522368
